# Human in the Loop con Microsoft Agent Framework

## Descripción

En este ejercicio, aprenderás a implementar el patrón **Human in the Loop** (HITL) para requerir aprobación humana antes de ejecutar acciones sensibles o críticas.

### Casos de uso:
- Aprobación de pagos o transferencias bancarias
- Confirmación de envío de emails
- Validación de cambios en bases de datos de producción
- Autorización de publicación de contenido
- Aprobación de órdenes de compra

## Configuración inicial

### Cargar librerías

Importamos las bibliotecas necesarias del Agent Framework.

In [ ]:
import os
from typing import Annotated
from pydantic import Field
from agent_framework import (
    ai_function,
    ChatMessage,
    Role,
    TextContent,
    FunctionCallContent,
    FunctionResultContent,
    FunctionApprovalRequestContent
)
from agent_framework.openai import OpenAIChatClient

# GitHub Models client configuration
MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

---

## Ejercicio: Sistema de Aprobación de Pagos

Crearemos un agente que puede procesar pagos, pero **siempre requiere aprobación humana** antes de ejecutar la transacción.

### Paso 1: Definir la función con aprobación requerida

Usamos el decorador `@ai_function` con `approval_mode="always_require"` para indicar que esta función necesita aprobación del usuario antes de ejecutarse.

In [ ]:
@ai_function(approval_mode="always_require")
def procesar_pago(
    destinatario: Annotated[str, Field(description="Nombre del beneficiario del pago")],
    importe: Annotated[float, Field(description="Cantidad a transferir en euros")],
    concepto: Annotated[str, Field(description="Descripción o concepto del pago")],
) -> str:
    """Procesa un pago bancario al destinatario especificado."""
    return f"Pago de {importe:.2f}€ procesado exitosamente a {destinatario}. Concepto: {concepto}"

print("Función de pago definida con approval_mode='always_require'")

### Paso 2: Crear el agente

Configuramos el agente con la función de pago que requiere aprobación.

In [ ]:
agent = OpenAIChatClient(
    model_id=MODEL_NAME,
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.github.ai/inference"
).create_agent(
    instructions="Eres un asistente financiero que ayuda a procesar pagos. Cuando el usuario solicite un pago, usa la función procesar_pago.",
    tools=procesar_pago
)

print("Agente creado correctamente")

### Paso 3: Ejecutar el agente y detectar solicitud de aprobación

Cuando el agente intenta llamar a una función con `approval_mode="always_require"`, la ejecución se pausa y devuelve un `FunctionApprovalRequestContent` que contiene los detalles de la función que se quiere ejecutar.

In [ ]:
# Solicitar un pago
user_request = "Necesito hacer un pago de 500 euros a Proveedor ABC por servicios de consultoría"

print(f"Usuario: {user_request}\n")
print("-" * 50)

# Primera ejecución - el agente intentará llamar a la función
async for update in agent.run_stream(user_request):
    pass

# Capturar y mostrar la solicitud de aprobación
if update.user_input_requests:
    approval_request = update.user_input_requests[0]

    print(f"\nTipo de solicitud: {type(approval_request).__name__}")

    print("\nAPROBACIÓN REQUERIDA")
    print(f"   Función: {approval_request.function_call.name}")
    print(f"   Argumentos: {approval_request.function_call.arguments}")

### Paso 4: Aprobar y completar la ejecución

Ahora pedimos la aprobación del usuario y continuamos la ejecución del agente.

La respuesta de aprobación se crea usando `create_response(True)` para aprobar o `create_response(False)` para rechazar.

In [ ]:
# Pedimos la aprobación al usuario
user_response = input("¿Desea aprobar el pago? (s/n): ").strip().lower()
user_approval = user_response == 's'

print(f"Usuario: {'Apruebo' if user_approval else 'Rechazo'} el pago\n")
print("-" * 50)

# Crear mensaje de aprobación
approval_message = ChatMessage(
    role=Role.USER,
    contents=[approval_request.create_response(user_approval)]
)

# Continuar la ejecución con el historial completo
full_response = ""
async for update in agent.run_stream([
    user_request,
    ChatMessage(role=Role.ASSISTANT, contents=[approval_request]),
    approval_message
]):
    for content in update.contents:
        if isinstance(content, TextContent):
            full_response += content.text
        elif isinstance(content, FunctionResultContent):
            print(f"Resultado: {content.result}")

# Mostrar la respuesta completa al final
if full_response:
    print(f"Agente: {full_response}")